In [1]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm
import warnings

# Ignore warnings throughout the code
warnings.filterwarnings("ignore")

In [3]:
db = sqlite3.connect('data/healthcare_bluebook.sqlite')

query = "SELECT * FROM hop"

hop_sqlite = pd.read_sql(query, db)

db.close()

In [67]:
db = sqlite3.connect('data/healthcare_bluebook.sqlite')

query = "SELECT * FROM npi"

npi_sqlite = pd.read_sql(query, db)

db.close()

In [15]:
db = sqlite3.connect('data/healthcare_bluebook.sqlite')

query = "SELECT * FROM taxonomy_classification"

taxonomy_sqlite = pd.read_sql(query, db)

db.close()

In [7]:
db = sqlite3.connect('data/healthcare_bluebook.sqlite')

query = "SELECT * FROM cbsa WHERE cbsa = 34980"

cbsa = pd.read_sql(query, db)

db.close()

In [124]:
db = sqlite3.connect('data/healthcare_bluebook.sqlite')

query = """
SELECT 
    h.from_npi,
    h.to_npi,
    h.patient_count,
    h.transaction_count,
    h.average_day_wait,
    n.provider_first_line_business_practice_location_address AS address,
    n.provider_organization_name_legal_business_name AS to_npi_name,
    n.provider_business_practice_location_address_city_name AS to_npi_city,
    n.provider_business_practice_location_address_state_name AS state,
    t.grouping AS to_npi_grouping,
    t.classification AS to_npi_classification
FROM hop AS h
INNER JOIN npi AS n
ON h.to_npi = n.npi
INNER JOIN taxonomy_classification AS t
ON t.code = n.healthcare_provider_taxonomy_code
WHERE n.entity_type_code = '2'
    AND to_npi_city = 'NASHVILLE'
    AND state = 'TN'
"""

to_npi_info = pd.read_sql(query, db)

db.close()

In [125]:
internal_med = to_npi_info[to_npi_info['to_npi_classification'] == 'Internal Medicine']
internal_med_average_wait = internal_med.groupby(['address', 'to_npi_name'])['average_day_wait'].mean().reset_index()
internal_med_patient_count = internal_med.groupby(['address', 'to_npi_name'])['patient_count'].sum().sort_values(ascending = False).reset_index()

In [73]:
internal_med_merge = pd.merge(internal_med_average_wait, internal_med_patient_count)
internal_med_merge.corr()

,average_day_wait,patient_count
average_day_wait,1.000000,0.235922
patient_count,0.235922,1.000000


In [74]:
internal_med_merge.sort_values(by=['patient_count'], ascending = False)

,address,to_npi_name,average_day_wait,patient_count
0,1301 22ND AVE S,VANDERBILT UNIVERSITY MEDICAL CENTER,24.828848,1166496
11,2011 MURPHY AVE,SAINT THOMAS MEDICAL PARTNERS,28.407259,1026115
9,2004 HAYES ST STE 800,TENNESSEE ONCOLOGY PLLC,30.431977,483887
27,28 WHITE BRIDGE RD,"NEPHROLOGY ASSOCIATES, PC",25.638585,277413
25,250 25TH AVE N,TENNESSEE ONCOLOGY PLLC,33.267812,267287
16,222 22ND AVE N,HERITAGE MEDICAL ASSOCIATES PC,32.740417,264312
19,2400 PATTERSON ST,COLUMBIA MEDICAL GROUP - THE FRIST CLINIC INC,26.978461,162087
18,2300 PATTERSON ST,MARATHON INPATIENT SERVICES PLLC,19.068303,83333
48,4230 HARDING RD,TENNESSEE ONCOLOGY PLLC,27.897310,19964
2,1414 COUNTY HOSPITAL RD,"PHYSICIAN SERVICES OF MIDDLE TENNESSEE, LLC",21.177131,19866


In [75]:
gen_acute_care = to_npi_info[to_npi_info['to_npi_classification'] == 'General Acute Care Hospital']
gen_acute_care_average_wait = gen_acute_care.groupby(['address', 'to_npi_name'])['average_day_wait'].mean().reset_index()
gen_acute_care_patient_count = gen_acute_care.groupby(['address', 'to_npi_name'])['patient_count'].sum().sort_values(ascending = False).reset_index()

In [76]:
gen_acute_care_merge = pd.merge(gen_acute_care_average_wait, gen_acute_care_patient_count)
gen_acute_care_merge.corr()

,average_day_wait,patient_count
average_day_wait,1.000000,0.767389
patient_count,0.767389,1.000000


In [77]:
gen_acute_care_merge.sort_values(by=['patient_count'], ascending = False)

,address,to_npi_name,average_day_wait,patient_count
0,1211 MEDICAL CENTER DRIVE,VANDERBILT UNIVERSITY MEDICAL CENTER,24.756911,771842
5,2300 PATTERSON ST,"HCA HEALTH SERVICES OF TENNESSEE, INC.",22.935219,362112
8,4220 HARDING RD,SAINT THOMAS WEST HOSPITAL,23.545144,269061
3,2000 CHURCH ST,SAINT THOMAS WEST HOSPITAL,23.436839,188438
6,3441 DICKERSON PIKE,HTI MEMORIAL HOSPITAL CORPORATION,20.664788,147367
7,391 WALLACE RD,"HCA HEALTH SERVICES OF TENNESSEE, INC.",18.510466,72442
2,1818 ALBION ST,NASHVILLE GENERAL HOSPITAL,15.232163,10887
1,1601 23RD AVE S,VANDERBILT UNIVERSITY MEDICAL CENTER,15.323611,7275
4,2000 CHURCH ST,SETON CORPORATION,11.547765,1409


In [100]:
radiology = to_npi_info[to_npi_info['to_npi_classification'] == 'Radiology']
radiology_average_wait = radiology.groupby(['address', 'to_npi_name'])['average_day_wait'].mean().reset_index()
radiology_patient_count = radiology.groupby(['address', 'to_npi_name'])['patient_count'].sum().sort_values(ascending = False).reset_index()

In [101]:
radiology_merge = pd.merge(radiology_average_wait, radiology_patient_count)
radiology_merge.corr()

,average_day_wait,patient_count
average_day_wait,1.000000,0.076081
patient_count,0.076081,1.000000


In [102]:
radiology_merge.sort_values(by=['patient_count'], ascending = False)

,address,to_npi_name,average_day_wait,patient_count
4,210 25TH AVENUE NORTH,RADIOLOGY ALLIANCE PC,26.326217,964905
7,28 WHITE BRIDGE ROAD,MIDDLE TENNESSEE IMAGING LLC,34.007453,143607
1,2004 HAYES ST STE 800,"TENNESSEE ONCOLOGY,PLLC",27.095801,35172
9,719 THOMPSON LN,"ONE HUNDRED OAKS IMAGING,LLC",29.709173,21178
0,1909 ACKLEN AVE,"VANDERBILT IMAGING SERVICES, LLC",27.877690,16714
10,843 RODNEY DR,CCF TENNESSEE MEDICAL SERVICES PC,33.873196,16331
3,2018 MURPHY AVE,"TENNESSEE ONCOLOGY,PLLC",27.739032,12570
8,330 23RD AVE N,"TRISTAR BONE MARROW TRANSPLANT, LLC",13.868038,1760
6,2410 PATTERSON ST,"TRISTAR RADIATION ONCOLOGY, LLC",19.614000,781
5,2201 MURPHY AVE,INNER VISION WOMENS ULTRASOUND PLC,11.567000,537


In [103]:
pharmacy = to_npi_info[to_npi_info['to_npi_classification'] == 'Pharmacy']
pharmacy_average_wait = pharmacy.groupby(['address', 'to_npi_name'])['average_day_wait'].mean().reset_index()
pharmacy_patient_count = pharmacy.groupby(['address', 'to_npi_name'])['patient_count'].sum().sort_values(ascending = False).reset_index()

In [104]:
pharmacy_merge = pd.merge(pharmacy_average_wait, pharmacy_patient_count)
pharmacy_merge.corr()

,average_day_wait,patient_count
average_day_wait,1.000000,-0.372167
patient_count,-0.372167,1.000000


In [105]:
pharmacy_merge.sort_values(by=['patient_count'], ascending = False)

,address,to_npi_name,average_day_wait,patient_count
0,1211 MEDICAL CENTER DR,VANDERBILT UNIVERSITY MEDICAL CENTER,22.581037,12912
15,624 GRASSMERE PARK,"OPTION CARE INFUSION SERVICES, LLC",24.827087,4280
3,1910 CHURCH ST,"CONTINUUMRX, INC.",20.286056,2303
4,2108 CHARLOTTE AVE,TENNESSEE ONCOLOGY PLLC,20.846536,1662
12,4230 HARDING PIKE,SAINT THOMAS WEST HOSPITAL,15.649462,902
8,309C 22ND AVE N,WALGREEN CO,14.313429,886
1,1215 21ST AVE S,VANDERBILT UNIVERSITY MEDICAL CENTER,22.464556,839
7,2970 SIDCO DR,"CORAM ALTERNATE SITE SERVICES, INC",23.237133,790
5,217 W MAPLEWOOD LANE,"I V SOLUTIONS, INC.",20.175706,738
14,4709 NOLENSVILLE PIKE,TENNESSEE CVS PHARMACY LLC,34.401333,91


In [109]:
rehab = to_npi_info[to_npi_info['to_npi_classification'] == 'Rehabilitation Hospital']
rehab_average_wait = rehab.groupby(['address', 'to_npi_name'])['average_day_wait'].mean().reset_index()
rehab_patient_count = rehab.groupby(['address', 'to_npi_name'])['patient_count'].sum().sort_values(ascending = False).reset_index()

In [110]:
rehab_merge = pd.merge(rehab_average_wait, rehab_patient_count)
rehab_merge.corr()

,average_day_wait,patient_count
average_day_wait,NaN,NaN
patient_count,NaN,NaN


In [111]:
rehab_merge.sort_values(by=['patient_count'], ascending = False)

,address,to_npi_name,average_day_wait,patient_count
0,2201 CHILDRENS WAY,VANDERBILT STALLWORTH REHABILITATION HOSPITAL LP,25.456762,4247


In [113]:
home_health = to_npi_info[to_npi_info['to_npi_classification'] == 'Home Health']
home_health_average_wait = home_health.groupby(['address', 'to_npi_name'])['average_day_wait'].mean().reset_index()
home_health_patient_count = home_health.groupby(['address', 'to_npi_name'])['patient_count'].sum().sort_values(ascending = False).reset_index()

In [114]:
home_health_merge = pd.merge(home_health_average_wait, home_health_patient_count)
home_health_merge.corr()

,average_day_wait,patient_count
average_day_wait,1.000000,-0.414732
patient_count,-0.414732,1.000000


In [116]:
home_health_merge.sort_values(by=['patient_count'], ascending = False)

,address,to_npi_name,average_day_wait,patient_count
2,1451 ELM HILL PIKE,"WILLOWBROOK HOME HEALTH CARE AGENCY, INC.",37.436944,8329
8,3443 DICKERSON PIKE STE 390,"SUNCREST HEALTHCARE OF MIDDLE TN, LLC",36.799242,7393
7,2601 ELM HILL PIKE STE AB&C,"AMEDISYS TENNESSEE, LLC",35.435317,6818
6,2525 PERIMETER PLACE,"LHC HOMECARE OF TENNESSEE, LLC",33.333850,4911
10,618 GRASSMERE PARK STE 5,INNOVATIVE SENIOR CARE HOME HEALTH OF NASHVILL...,32.721415,4815
4,2000 GLEN ECHO RD,"AMEDISYS TENNESSEE, LLC",38.852341,4349
5,2120 BELCOURT AVE,"VANDERBILT HOME CARE SERVICES, LLC",31.380333,2131
1,1321 MURFREESBORO PIKE STE 512,GENTIVA CERTIFIED HEALTHCARE CORP.,35.574083,2040
3,2 INTERNATIONAL PLAZA,"HOME HEALTH CARE OF MIDDLE TENNESSEE, LLC",39.067217,1842
9,4015 TRAVIS DR,"VIP HOME NURSING & REHABILITATION SERVICE, LLC",38.842846,737


In [117]:
clinic_center = to_npi_info[to_npi_info['to_npi_classification'] == 'Clinic/Center']
clinic_center_average_wait = clinic_center.groupby(['address', 'to_npi_name'])['average_day_wait'].mean().reset_index()
clinic_center_patient_count = clinic_center.groupby(['address', 'to_npi_name'])['patient_count'].sum().sort_values(ascending = False).reset_index()

In [118]:
clinic_center_merge = pd.merge(clinic_center_average_wait, clinic_center_patient_count)
clinic_center_merge.corr()

,average_day_wait,patient_count
average_day_wait,1.000000,0.339089
patient_count,0.339089,1.000000


In [122]:
test = clinic_center_merge.sort_values(by=['patient_count'], ascending = False)
test

,address,to_npi_name,average_day_wait,patient_count
46,3443 DICKERSON PIKE,MIDDLE TENNESSEE PULMONARY ASSOCIA,27.983658,63574
15,1921 RANSOM PL,"CENTERSTONE OF TENNESSEE, INC.",23.704651,29302
31,2801 CHARLOTTE AVE,UROLOGY SURGERY CENTER LP,15.942024,10068
2,1005 DR. D. B. TODD BLVD,MEHARRY MEDICAL COLLEGE,16.678955,9772
57,4230 HARDING PIKE,SAINT THOMAS CAMPUS SURGICARE LP,14.126280,9458
...,...,...,...,...
47,3443 DICKERSON PIKE STE 660,"CLARK FAMILY PRACTICE, PLLC",0.000000,31
77,865 BELLEVUE RD,MESECK FAMILY CHIROPRACTIC LLC,0.000000,28
70,5515 EDMONDSON PIKE,"HICKORY MEDICAL ADVISORS, PC",0.000000,27
6,107 CHARLES E DAVIS BLVD,"UNITED NEIGHBORHOOD HEALTH SERVICES, INC",1.500000,23


In [123]:
test[test['to_npi_name'].str.contains("Vanderbilt", case = False, na = False)]

,address,to_npi_name,average_day_wait,patient_count
32,2906 FOSTER CREIGHTON DR STE 100,VANDERBILT UNIVERSITY MEDICAL CENTER,15.378471,1013
63,4525 HARDING RD,"VANDERBILT IMAGING SERVICES, LLC",9.319000,543
17,20 RACHEL DRIVE,VANDERBILT UNIVERSITY MEDICAL CENTER,7.237167,267


In [98]:
# Average Day Wait for Vanderbilt By Specialty
nashville_cbsa[nashville_cbsa['to_npi_name'].str.contains("Vanderbilt", case = False, na = False)].groupby(['to_npi_name', 'to_npi_classification'])['patient_count'].sum().sort_values(ascending = False)

to_npi_name                                       to_npi_classification      
VANDERBILT UNIVERSITY MEDICAL CENTER              Internal Medicine              1166496
                                                  General Acute Care Hospital     779117
VANDERBILT IMAGING SERVICES, LLC                  Radiology                        16714
VANDERBILT UNIVERSITY MEDICAL CENTER              Pharmacy                         13751
VANDERBILT STALLWORTH REHABILITATION HOSPITAL LP  Rehabilitation Hospital           4247
VANDERBILT HOME CARE SERVICES, LLC                Home Health                       2131
VANDERBILT UNIVERSITY MEDICAL CENTER              Clinic/Center                     1280
VANDERBILT IMAGING SERVICES, LLC                  Clinic/Center                      543
Name: patient_count, dtype: int64